In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import uproot
import scipy.stats as st
from scipy.special import beta
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random
import os
import shutil
from pathlib import Path
from subprocess import check_output

from config import *

random.seed(30)

data_folder = root_folder + 'user.eschanet.allTrees_v2_0_2_signal_1Lbb_skim.root'

In [2]:
root = uproot.open(data_folder)

In [3]:
signal_id = []
count = 0

for n in root.keys():
        if 'nosys' not in str(n).lower():
            n = str(n)
            signal_id.append('_'.join(n.split('Wh_hbb_')[1].split('_')[0:2]))            

In [4]:
unique_name = np.unique(signal_id)

In [5]:
syst = ['JET_JER_EffectiveNP_1',
'JET_JER_EffectiveNP_2',
'JET_JER_EffectiveNP_3',
'JET_JER_EffectiveNP_4',
'JET_JER_EffectiveNP_5',
'JET_JER_EffectiveNP_6',
'JET_JER_EffectiveNP_7',
'JET_JER_EffectiveNP_8',
'JET_JER_EffectiveNP_9',
'JET_JER_EffectiveNP_10',
'JET_JER_EffectiveNP_11',
'JET_JER_EffectiveNP_12restTerm',
'JET_JER_DataVsMC',
'JET_BJES_Response',
'JET_EffectiveNP_Mixed1',
'JET_EffectiveNP_Mixed2',
'JET_EffectiveNP_Mixed3',
'JET_EffectiveNP_Detector1',
'JET_EffectiveNP_Detector2',
'JET_EffectiveNP_Modelling1',
'JET_EffectiveNP_Modelling2',
'JET_EffectiveNP_Modelling3',
'JET_EffectiveNP_Modelling4',
'JET_EffectiveNP_Statistical1',
'JET_EffectiveNP_Statistical2',
'JET_EffectiveNP_Statistical3',
'JET_EffectiveNP_Statistical4',
'JET_EffectiveNP_Statistical5',
'JET_EffectiveNP_Statistical6',
'JET_EtaIntercalibration_Modelling',
'JET_EtaIntercalibration_NonClosure_highE',
'JET_EtaIntercalibration_NonClosure_negEta',
'JET_EtaIntercalibration_NonClosure_posEta',
'JET_EtaIntercalibration_TotalStat',
'JET_Flavor_Composition',
'JET_Flavor_Response',
'JET_Pileup_OffsetMu',
'JET_Pileup_OffsetNPV',
'JET_Pileup_PtTerm',
'JET_Pileup_RhoTopology',
'JET_PunchThrough_MC16',
'JET_SingleParticle_HighPt',
'EG_RESOLUTION_ALL',
'EG_SCALE_ALL',
'EG_SCALE_AF2',
'EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR',
'EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR',
'EL_EFF_Iso_TOTAL_1NPCOR_PLUS_UNCOR',
'MUON_ID',
'MUON_MS',
'MUON_SCALE',
'MUON_SAGITTA_RHO',
'MUON_SAGITTA_RESBIAS',
'MUON_EFF_RECO_STAT',
'MUON_EFF_RECO_SYS',
'MUON_EFF_RECO_STAT_LOWPT',
'MUON_EFF_RECO_SYS_LOWPT',
'MUON_EFF_ISO_STAT',
'MUON_EFF_ISO_SYS',
'MUON_EFF_BADMUON_STAT',
'MUON_EFF_BADMUON_SYS',
'MUON_EFF_TTVA_STAT',
'MUON_EFF_TTVA_SYS',
'FT_EFF_B_systematics',
'FT_EFF_C_systematics',
'FT_EFF_Light_systematics',
'FT_EFF_extrapolation',
'FT_EFF_extrapolation_from_charm',
'MET_SoftTrk_Scale',
'MET_SoftTrk_ResoPara',
'MET_SoftTrk_ResoPerp',
'jvtWeightJET_JvtEfficiency',
'pileupWeight']

In [6]:
signal_dict = {}

for n in unique_name:
    for names in root.keys():
        if n in str(names):
            if 'nosys' not in str(names).lower():
                if 'up' in str(names).lower():
                    if ('_'.join(str(names).split('_')[5:])).split('__')[0] in syst:
                        try:
                            signal_dict[n].append(str(names).split(";")[0].split("'")[1])
                        except:
                            signal_dict[n] = [str(names).split(";")[0].split("'")[1]]          

In [7]:
columns = ['met', 'mt', 'mbb', 'mct2',
        'mlb1', 'nJet30', 'lep1Pt', 'nBJet30_MV2c10',
           'genWeight','eventWeight', 'pileupWeight','leptonWeight','bTagWeight','jvtWeight',
        'trigMatch_metTrig', 'nLep_signal']

In [11]:
unique_name = ['600p0_50p0']

In [12]:
entrysteps=3000000
tot = 0

for folder in unique_name:
    dir = numpy_sig_syst_up + '{}'.format(folder)
    if os.path.exists(dir):
#         shutil.rmtree(dir)
        continue
    else:
        os.makedirs(dir)

    for name in signal_dict[folder]:

        events = uproot.open(data_folder)[name]
        array = events.lazyarray('met')

        print('lunghezza array', len(array))
        file_split = len(array)//entrysteps
        start_name_file = 0
        entrystart = start_name_file*entrysteps

        print(name)

        batches = events.iterate(columns, entrystart=entrystart,
                               entrysteps=entrysteps, 
                                   outputtype=pd.DataFrame)

        for ix in range(start_name_file, file_split+1):

            print(ix)    
            batch = next(batches)
            print('adding luminosity')
            batch['luminosity'] = 139000
            print(len(batch))

            batch = batch[batch['nLep_signal'].astype(int)==1]
            print('after signal {}'.format(len(batch)))

            batch = batch[batch['trigMatch_metTrig'].astype(int)==1]
            print('after trig {}'.format(len(batch)))

            batch = batch[((batch['nBJet30_MV2c10']>=1)&(batch['nBJet30_MV2c10']<4))]
            print('after bjet {}'.format(len(batch)))

            batch = batch[((batch['met']>=220)&(batch['met']<=1000))]         
            print('after met {}'.format(len(batch)))

            batch = batch[((batch['mt']>=50)&(batch['mt']<=1000))]         
            print('after mt {}'.format(len(batch)))

            batch = batch[((batch['mbb']>=100)&(batch['mbb']<350))]  
            print('after mbb {}'.format(len(batch)))       

            batch = batch[((batch['mct2']>=100)&(batch['mct2']<1000))] 
            print('after mct2 {}'.format(len(batch)))
            
            batch = batch[((batch['mlb1']>=0)&(batch['mlb1']<1000))] 
            print('after mct2 {}'.format(len(batch)))
            
            batch = batch[((batch['lep1Pt']>=0)&(batch['lep1Pt']<1000))] 
            print('after mct2 {}'.format(len(batch)))

            if len(batch) > 0:


                batch['weight'] = batch.apply(lambda row: row['genWeight']*row['eventWeight']*row['pileupWeight']*
                                     row['leptonWeight']*row['bTagWeight']*row['jvtWeight']*row['luminosity'], axis=1)

                batch_fin = batch.iloc[:,:8]

                batch_fin['weight'] = batch['weight']

                batch_fin = batch_fin[['met', 'mt', 'mbb', 'mct2',
                'mlb1','lep1Pt', 'nJet30', 'nBJet30_MV2c10', 'weight']]

                tot = tot + len(batch)
                print('tot = {}'.format(tot))
                print("\x1b[31m\"saving {}_{}""\x1b[0m".format(name,ix))
                np.save(numpy_sig_syst_up + '{}/{}.npy'.format(folder,name), batch_fin.values)


lunghezza array 645
C1N2_Wh_hbb_600p0_50p0_EG_RESOLUTION_ALL__1up
0
adding luminosity
645
after signal 645
after trig 634
after bjet 634
after met 634
after mt 626
after mbb 504
after mct2 486
after mct2 478
after mct2 478
tot = 478
"saving C1N2_Wh_hbb_600p0_50p0_EG_RESOLUTION_ALL__1up_0
lunghezza array 648
C1N2_Wh_hbb_600p0_50p0_EG_SCALE_AF2__1up
0
adding luminosity
648
after signal 648
after trig 637
after bjet 637
after met 637
after mt 629
after mbb 506
after mct2 488
after mct2 480
after mct2 480
tot = 958
"saving C1N2_Wh_hbb_600p0_50p0_EG_SCALE_AF2__1up_0
lunghezza array 646
C1N2_Wh_hbb_600p0_50p0_EG_SCALE_ALL__1up
0
adding luminosity
646
after signal 646
after trig 635
after bjet 635
after met 635
after mt 627
after mbb 505
after mct2 487
after mct2 479
after mct2 479
tot = 1437
"saving C1N2_Wh_hbb_600p0_50p0_EG_SCALE_ALL__1up_0
lunghezza array 648
C1N2_Wh_hbb_600p0_50p0_JET_BJES_Response__1up
0
adding luminosity
648
after signal 648
after trig 637
after bjet 637
after met 637
a